# Homework

In [1]:
import numpy as np
import time

from joblib import Parallel, delayed, parallel_backend

In [21]:
def match_timestamps(timestamps_30fps, timestamps_60fps):
    """
    Let's say we have two cameras capturing the same scene. 
    One camera's frame rate is 60, antoher's is 30. However, due to a high CPU or 
    hard drive load the actual fps might differ from 30 and 60.
    
    This function matches the frames from the first camera to the second one, meaning
    that for each timestamp in timestamps_60fps it finds the index of closest one in timestamps_30fps.
    
    Inputs:
        - timestamps_30fps: sorted np.ndarray, dtype=np.floa64. Timestamps of the 30 fps camera.
        - timestamps_60fps: sorted np.ndarray, dtype=np.floa64. Timestamps of the 60 fps camera. 
    Outputs:
        - idxs: np.ndarray, dtype=np.int32. Indexes of timestamps matching.

    Example:
        - timestamps_30fps = np.array([0, 0.033, 0.066], dtype=np.float64)
        - timestamps_60fps = np.array([0, 0.016, 0.032, 0.048, 0.065], dtype=np.float64)
        - idxs = np.array([0, 0, 1, 1, 2], dtype=np.int32)
    
    This function must be as fast as possible on CPU from both algorithmic and Python-wise implementation perspectives.
    It must provide the correct result and must not fail on any realization of the described input. 
    You may use ANY lib or method you want up to writing a C++ wrapping callable from Python. 
    Consider you have multiple CPU cores.
    Send the best implementation of this function to asshilov@yandex.com or to tg @a_team2 before 23:59 24.11 
    in .py or .ipynb formats.
    Good luck!
    """
    idxs = np.ones(len(timestamps_60fps), dtype=np.int32) # replace this with your code
    return idxs

def get_sample_timestamps(fps, video_length_sec):
    timestamps = np.linspace(time.time(), time.time() + video_length_sec, video_length_sec * fps)
    timestamps += np.random.randn(len(timestamps)) / fps # emulate high CPU of drive load
    timestamps = np.sort(timestamps) # despite the load, timestamps have to be sorted
    return timestamps

video_length_sec = 1000 # optimize the implementation for the lengths in range 1000-4000 seconds
timestamps_30fps = get_sample_timestamps(30, video_length_sec)
timestamps_60fps = get_sample_timestamps(60, video_length_sec)
match_timestamps(timestamps_30fps, timestamps_60fps)

array([1, 1, 1, ..., 1, 1, 1], dtype=int32)

In [19]:
def match_timestamps_block(t30, t60):
    if t30.shape[0] == 0 or t60.shape[0] == 0:
        return np.array([])

    left_ptr = 0
    if not t60[0] < t30[0]:
        le, ri = 0, len(t30)
        while le + 1 < ri:
            m = (le + ri) // 2
            if t30[m] < t60[0]:
                le = m
            else:
                ri = m
        left_ptr = le
    right_ptr = 0

    ans = np.zeros(t60.shape[0])
    while right_ptr < t60.shape[0]:
        ans[right_ptr] = left_ptr if left_ptr + 1 == t30.shape[0] or abs(t60[right_ptr] - t30[left_ptr]) < abs(t60[right_ptr] - t30[left_ptr + 1]) else left_ptr + 1
        right_ptr += 1
        if right_ptr == t60.shape[0]:
            break
        while left_ptr < t30.shape[0] and t30[left_ptr] < t60[right_ptr]:
            left_ptr += 1
        left_ptr -= 1
   
    return ans

match_timestamps_block(np.array([0, 0.033, 0.066]), np.array([0, 0.016, 0.032, 0.048, 0.065]))

array([0., 0., 1., 1., 2.])

In [22]:
batch_size = 100
n_times = 10

t0 = time.time()
for _ in range(n_times):
    with parallel_backend(backend="loky", n_jobs=8):
        par_res = Parallel()(delayed(match_timestamps_block)(
            timestamps_30fps, 
            timestamps_60fps[batch_start:min(batch_start + batch_size, timestamps_60fps.shape[0])])
            for batch_start in range(0, timestamps_60fps.shape[0], batch_size))

    ans = np.array([])
    for res in par_res:
        ans = np.append(ans, res)
print(f"Total time: {(time.time() - t0) / n_times:.3f} sec")

ans.shape

Total time: 0.381 sec


(60000,)

# UPDATE (18.12)

In [25]:
import numba

In [26]:
@numba.jit(parallel=True)
def match_timestamps(t30, t60):
    ans = np.zeros(t60.shape[0])

    num_threads = numba.get_num_threads()
    batch_size = (t60.shape[0] // num_threads) + 1

    def match_timestamps_block(from_pos, to_pos):
        if t30.shape[0] == 0 or from_pos == to_pos:
            return

        left_ptr = 0
        if not t60[from_pos] < t30[0]:
            le, ri = 0, len(t30)
            while le + 1 < ri:
                m = (le + ri) // 2
                if t30[m] < t60[from_pos]:
                    le = m
                else:
                    ri = m
            left_ptr = le

        right_ptr = from_pos
        while right_ptr < to_pos:
            ans[right_ptr] = left_ptr if left_ptr + 1 == t30.shape[0] or np.abs(t60[right_ptr] - t30[left_ptr]) < np.abs(t60[right_ptr] - t30[left_ptr + 1]) else left_ptr + 1
            right_ptr += 1
            if right_ptr == to_pos:
                break
            while left_ptr < t30.shape[0] and t30[left_ptr] < t60[right_ptr]:
                left_ptr += 1
            left_ptr -= 1

    for i in numba.prange(num_threads):
        from_pos = i * batch_size
        to_pos = min(from_pos + batch_size, t60.shape[0])
        match_timestamps_block(from_pos, to_pos)
        
    return ans

In [29]:
n_times = 100
video_length_sec = 4000
timestamps_30fps = get_sample_timestamps(30, video_length_sec)
timestamps_60fps = get_sample_timestamps(60, video_length_sec)
match_timestamps(timestamps_30fps, timestamps_60fps)

t0 = time.time()
for _ in range(n_times):
    match_timestamps(timestamps_30fps, timestamps_60fps)
print(f"Total time: {(time.time() - t0) / n_times:.5f} sec")

Total time: 0.00075 sec
